In [1]:
import Pkg

Pkg.add("Interpolations")
Pkg.add("PyPlot")
Pkg.add("PyCall")
Pkg.add("OrdinaryDiffEq")
Pkg.add("YAML")
Pkg.add("DelimitedFiles")
Pkg.add("CSV")
Pkg.add("HDF5")
Pkg.add("StructArrays")
Pkg.add("Random")

using Interpolations, PyCall, OrdinaryDiffEq,
YAML, DelimitedFiles, CSV, HDF5, StructArrays, Random, 
NBInclude, PyPlot

ct = pyimport("cantera")

   Updating registry at `~/.julia/registries/General`


######################################################################## 100.0%


  Resolving package versions...


No Changes to `~/GCresearch/rocketemissions/Project.toml`
No Changes to `~/GCresearch/rocketemissions/Manifest.toml`
  Resolving package versions...


No Changes to `~/GCresearch/rocketemissions/Project.toml`
No Changes to `~/GCresearch/rocketemissions/Manifest.toml`
  Resolving package versions...


...

No Changes to `~/GCresearch/rocketemissions/Project.toml`
No Changes to `~/GCresearch/rocketemissions/Manifest.toml`
  Resolving package versions...


No Changes to `~/GCresearch/rocketemissions/Project.toml`
No Changes to `~/GCresearch/rocketemissions/Manifest.toml`
  Resolving package versions...


No Changes to `~/GCresearch/rocketemissions/Project.toml`
No Changes to `~/GCresearch/rocketemissions/Manifest.toml`
  Resolving package versions...


No Changes to `~/GCresearch/rocketemissions/Project.toml`
No Changes to `~/GCresearch/rocketemissions/Manifest.toml`


PyObject <module 'cantera' from '/opt/conda/envs/lae2020/lib/python3.8/site-packages/cantera/__init__.py'>

In [1]:
### COMPUTE u AND T: FUNCTIONS ###

function compute_y(u::AbstractVector, T::AbstractVector, Δψ::AbstractVector,
    R::AbstractFloat, p::AbstractFloat)
    return (2 * trapz((R / p) * T ./ u) .* Δψ) .^ 0.5
end

compute_y (generic function with 1 method)

In [8]:
### COMPUTE u AND T ###
# No chemistry

## Constants/Inputs ##
h = [16000] # altitude to caluclate plume properties at [m]
h_string = "16000"
n = 100 # steps in x/ϕ direction
s = 50 # steps in y/ψ direction
m = 1 # counter for test set to only test single altitude
Pr = 1.0 # prandtl number
Le = 1.0 # lewis number

# create spacing arrays for ϕ and ψ
Δϕ = 0.1 * ones(n) #step size in phi
for i = 2:n
    Δϕ[i] = 1.05 * Δϕ[i-1] #enlarge with each step by 1.05
end

Δψ = 0.1 * ones(s) #s vertical grid points
for i = 2:s
    Δψ[i] = 1.05 * Δψ[i-1] #enlarge with each step by 1.05
end

# Import rocket exit conditions from upstream (u, T, p)
u0 = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/u")# initial plume velocity [m/s]
u0 = convert(AbstractFloat, u0[2])
T0 = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/T") # initial plume temperature [K]
T0 = convert(AbstractFloat, T0[2])
p_all = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/P") #ambient pressure [Pa]
p = convert(AbstractFloat, p_all[2])

#curve fit #a = ambient vel [m/s] (speed of rocket) #NEED TO UPDATE W HORIZONTAL VELOCITY
u_a = 1.11849E-19 * big(h[m])^5 - 1.14814E-14 * big(h[m])^4 + 4.22542E-10 * big(h[m])^3 - 6.92322E-06 * big(h[m])^2 + 6.58761E-02 * big(h[m]) + 5.37920E+01
T_a = HDF5.h5read("/home/chinahg/GCresearch/rocketemissions/plot_data.h5", h_string * "m/T_a") # ambient atmospheric temperature [K]

#CHECK THAT IMPORTED CONDITIONS ARE CORRECT

## Initialize initial u and T conditions: at x/ϕ = 0, y/ψ = all
u_init = u0 .* ones(s) # array of length y/ψ filled with rcoket exit velocity value [m/s]
T_init = T0 .* ones(s) # array of length y/ψ filled with rocket exit temperature value [K]
R_init = 

# we need to define the initial plume geometry by filling the initial u and T arrays with ambient u and T values outside of the plume radius
radius = 1.147 # rocket exit plume radius [m]

# have to map ψ in y domain to map properties to the correct grid-points (inside or outside plume radius)
y_init = compute_y(u_init, T_init, Δψ, R, p) #REDERIVE

# if y is greater than the radius of the plume, assign the ambient conditions to that point
u_init[y_init.>radius] .= convert(AbstractFloat, u_a)
T_init[y_init.>radius] .= convert(AbstractFloat, T_a)

#R = 
#
#ambient = AmbientConditions(u_a, T_a, 1.0, 1.0, R, p, T0, u0)
#
##Solve for T and u at all steps: NO CHEMISTRY
#x, y, u, T, ϵ = solve_exhaust_flow(u_init, T_init, ambient, n, Δϕ, Δψ)
#x_save = x
#y_save = y
#T_save[m, :, :] = T
#u_save[m, :, :] = u

UndefVarError: UndefVarError: R not defined

In [ ]:
# Computing u and T independent of chemistry
x, y, u, T, ϵ = solve_exhaust_flow(u_init, T_init, ambient, n, Δϕ, Δψ)